In [81]:
import torch
import torch.nn as nn
from torch.distributions import normal
from torch.distributions import uniform
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.pyplot import figure
from pylab import rcParams
rcParams['figure.figsize'] = (20, 20) # set default size of plots
rcParams['image.interpolation'] = 'nearest'
rcParams['image.cmap'] = 'gray'

from models import Reservoir
from utils import sinMSE, descend
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Very slow, and bad implementation, and buggy, but this is an implementation of Haas' reservoir 2004

In [80]:
num_neurons = 50
connectivity = 0.2
spectral_radius = 1.0

dt = 0.5

r = Reservoir(num_neurons,connectivity,spectral_radius,init_pattern = 'random',feedback=True,bias=False,fb_scale=1,record_history=True)

num_warmup = 100
num_train = 100

# Let the network warm up
for step in range(num_warmup):
    r.forward()
    


# Training dataset
for s in range(num_train):
    targ = torch.sin(torch.tensor(s*dt))
    
    r.y = torch.tensor([targ])
    r.forward()
    
    
# Train it
readout_weight = torch.tensor(r.readout_w,requires_grad=True)

epochs = 400
lr=0.01

for e in range(epochs):
    x = torch.tensor(r.history['state_vectors'])[num_warmup:num_warmup+num_train,:]        
    y = torch.tensor(r.history['outputs'])[num_warmup:num_warmup+num_train,:]
    
    diff = y - torch.matmul(x,readout_weight)
    se = diff.pow(2)
    mse = torch.sum(se) / (num_train-num_warmup)
    
    mse.backward()
    
    
    grad = readout_weight.grad
 
    
    readout_weight = readout_weight - grad*lr
    readout_weight = torch.tensor(readout_weight,requires_grad=True)

print(mse)
    
r.readout_w = readout_weight


/home/alpinefunker/.local/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/alpinefunker/.local/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


tensor(nan, grad_fn=<DivBackward0>)
